In [1]:
import pandas as pd
import streamlit as st

In [2]:
import requests

base_url = "https://api.eia.gov/v2/petroleum/pnp/inpt2/data/"
res = requests.get(
    base_url,
    params={
        "api_key": st.secrets["EIA_KEY"],
        "frequency": "monthly",
        "data[0]": "value",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": 0,
        "length": 5000,
    },
)
data = res.json()
df = pd.DataFrame(data["response"]["data"])

In [10]:
df = pd.DataFrame(data["response"]["data"])
# Convert period to datetime
df["Date"] = pd.to_datetime(df["period"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Separate state-level rows
df_states = df[df["duoarea"].str.startswith("S")].copy()

# Separate main PADD-level rows
valid_padds = ["R10", "R20", "R30", "R40", "R50"]
df_padds = df[df["duoarea"].isin(valid_padds)].copy()


df_top_pivot = df_padds.pivot_table(
    index="Date", columns="duoarea", values="value", aggfunc="sum"
)
df_top_pivot = df_top_pivot.rename(
    columns={
        "R10": "PADD 1",
        "R20": "PADD 2",
        "R30": "PADD 3",
        "R40": "PADD 4",
        "R50": "PADD 5",
    }
)

df_top_pivot["US Total"] = (
    df_top_pivot["PADD 1"]
    + df_top_pivot["PADD 2"]
    + df_top_pivot["PADD 3"]
    + df_top_pivot["PADD 4"]
    + df_top_pivot["PADD 5"]
)

In [7]:
url = "https://api.eia.gov/v2/petroleum/sum/snd/data/"
params = {
    "api_key": st.secrets["EIA_KEY"],
    "frequency": "monthly",
    "data[0]": "value",
    "facets[duoarea][]": ["R10", "R20", "R30", "R40", "R50"],
    "facets[process][]": ["FPF"],
    "facets[product][]": ["EPC0"],
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000,
}
r = requests.get(url, params=params)
data = r.json()
df = pd.DataFrame(data["response"]["data"])

In [9]:
df

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2025-04,R10,PADD 1,EPC0,Crude Oil,FPF,Field Production,MCRFPP11,East Coast (PADD 1) Field Production of Crude ...,1493,MBBL
1,2025-04,R40,PADD 4,EPC0,Crude Oil,FPF,Field Production,MCRFPP42,Rocky Mountain (PADD 4) Field Production of Cr...,1018,MBBL/D
2,2025-04,R30,PADD 3,EPC0,Crude Oil,FPF,Field Production,MCRFPP32,Gulf Coast (PADD 3) Field Production of Crude ...,9883,MBBL/D
3,2025-04,R30,PADD 3,EPC0,Crude Oil,FPF,Field Production,MCRFPP31,Gulf Coast (PADD 3) Field Production of Crude ...,296477,MBBL
4,2025-04,R20,PADD 2,EPC0,Crude Oil,FPF,Field Production,MCRFPP22,Midwest (PADD 2) Field Production of Crude Oil...,1804,MBBL/D
...,...,...,...,...,...,...,...,...,...,...,...
4995,1983-09,R40,PADD 4,EPC0,Crude Oil,FPF,Field Production,MCRFPP42,Rocky Mountain (PADD 4) Field Production of Cr...,573,MBBL/D
4996,1983-09,R50,PADD 5,EPC0,Crude Oil,FPF,Field Production,MCRFPP52,West Coast (PADD 5) Field Production of Crude ...,2859,MBBL/D
4997,1983-09,R10,PADD 1,EPC0,Crude Oil,FPF,Field Production,MCRFPP11,East Coast (PADD 1) Field Production of Crude ...,2319,MBBL
4998,1983-09,R10,PADD 1,EPC0,Crude Oil,FPF,Field Production,MCRFPP12,East Coast (PADD 1) Field Production of Crude ...,77,MBBL/D


In [19]:
pl.DataFrame(df_top_pivot)

PADD 1,PADD 2,PADD 3,PADD 4,PADD 5,US Total
f64,f64,f64,f64,f64,f64
17727.0,-33175.0,13210.0,-3992.0,79300.0,73070.0
1204.0,-29282.0,63058.0,16511.0,-9117.0,42374.0
3246.0,-37002.0,72196.0,15905.0,-7940.0,46405.0
3204.0,-35750.0,42111.0,15351.0,-8484.0,16432.0
1664.0,-27767.0,38411.0,13230.0,-7417.0,18121.0
6177.0,-22232.0,49511.0,16693.0,5320.0,55469.0


In [20]:
url = "https://api.eia.gov/v2/petroleum/cons/wpsup/data/"
params = {
    "api_key": st.secrets["EIA_KEY"],
    "frequency": "weekly",
    "data[0]": "value",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000,
}
response = requests.get(url, params=params)
data = response.json()
df = pd.DataFrame(data["response"]["data"])

In [21]:
df

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2025-06-20,NUS,U.S.,EPPO4,Other Oils,VPP,Product Supplied,WWOUP_NUS_2,U.S. Product Supplied of Other Oils (Thousand ...,4854,MBBL/D
1,2025-06-20,NUS,U.S.,EPJK,Kerosene-Type Jet Fuel,VPP,Product Supplied,WKJUPUS2,U.S. Product Supplied of Kerosene-Type Jet Fue...,1706,MBBL/D
2,2025-06-20,NUS,U.S.,EPPR,Residual Fuel Oil,VPP,Product Supplied,WREUPUS2,U.S. Product Supplied of Residual Fuel Oil (Th...,167,MBBL/D
3,2025-06-20,NUS,U.S.,EPD0,Distillate Fuel Oil,VPP,Product Supplied,WDIUPUS2,U.S. Product Supplied of Distillate Fuel Oil (...,3794,MBBL/D
4,2025-06-20,NUS,U.S.,EPM0F,Finished Motor Gasoline,VPP,Product Supplied,WGFUPUS2,U.S. Product Supplied of Finished Motor Gasoli...,9688,MBBL/D
...,...,...,...,...,...,...,...,...,...,...,...
4995,2009-07-10,NUS,U.S.,EPLLPZ,Propane and Propylene,VPP,Product Supplied,WPRUP_NUS_2,U.S. Product Supplied of Propane and Propylene...,898,MBBL/D
4996,2009-07-10,NUS,U.S.,EPD0,Distillate Fuel Oil,VPP,Product Supplied,WDIUPUS2,U.S. Product Supplied of Distillate Fuel Oil (...,3439,MBBL/D
4997,2009-07-10,NUS,U.S.,EPPO4,Other Oils,VPP,Product Supplied,WWOUP_NUS_2,U.S. Product Supplied of Other Oils (Thousand ...,3226,MBBL/D
4998,2009-07-03,NUS,U.S.,EPPO4,Other Oils,VPP,Product Supplied,WWOUP_NUS_2,U.S. Product Supplied of Other Oils (Thousand ...,3181,MBBL/D


In [22]:
df["units"].unique()

array(['MBBL/D'], dtype=object)